<a href="https://colab.research.google.com/github/Tttthea/fashion_tp/blob/main/demo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title enviroment
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))
!pip install ninja gradio fbpca boto3 requests==2.23.0 urllib3==1.25.11
!pip install diffusers transformers scipy
!pip install "ipywidgets>=7,<8"

<IPython.core.display.Javascript object>

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 14.5 MB/s 
     |████████████████████████████████| 13.8 MB 71.5 MB/s 
     |████████████████████████████████| 132 kB 77.8 MB/s 
     |████████████████████████████████| 127 kB 82.0 MB/s 
     |████████████████████████████████| 2.3 MB 57.6 MB/s 
     |████████████████████████████████| 106 kB 84.5 MB/s 
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 278 kB 81.5 MB/s 
     |████████████████████████████████| 84 kB 4.4 MB/s 
     |████████████████████████████████| 84 kB 5.0 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 79 kB 10.1 MB/s 
     |████████████████████████████████| 10.3 MB 54.0 MB/s 
     |████████████████████████████████| 64 kB 1.9 MB/s 
     |████████████████████████████████| 80 kB 10.8 MB/s 
     |████████████████████████████████| 6

In [ ]:
#@title login hugging-face
from google.colab import output
from huggingface_hub import notebook_login
output.enable_custom_widget_manager()
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
#@title load stable diffusion
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

from PIL import Image  
import re
import os


model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"


pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16")
pipe = pipe.to(device)

<IPython.core.display.Javascript object>

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.63k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [ ]:
#@title create userInterface
import gradio as gr
from google.colab import drive
drive. mount('/content/drive')
%cd drive/My Drive/fashion
block = gr.Blocks()

def infer(prompt, samples):  
    generator = torch.Generator(device=device)
    promp =  prompt + "in white background"
    images_list = pipe(  
        [promp] * samples,
        # num_inference_steps=steps,
        # guidance_scale=scale,
        generator=generator,
    )

    images = []
    safe_image = Image.open(r'./img/dress/img_00000001.jpg')
    for i, image in enumerate(images_list.images):
        if(images_list["nsfw_content_detected"][i]):
            images.append(safe_image)
        else:
            images.append(image)
    return images

with block:
    with gr.Group():
        with gr.Box():

            with gr.Row().style(mobile_collapse=False, equal_height=True):  
                text = gr.Textbox(
                    label="Describe your requirement",
                    show_label=False,
                    max_lines=1,
                    placeholder="Describe your requirement",
                ).style(
                    border=(True, False, True, True),
                    rounded=(True, False, False, True),
                    container=False,
                )
                btn = gr.Button("Generate image").style(
                    margin=False,
                    rounded=(False, True, True, False),
                )
        gallery = gr.Gallery(
            label="Generated clothes", show_label=False, elem_id="gallery"
        ).style(grid=[2], height="auto")

        advanced_button = gr.Button("Advanced options", elem_id="advanced-btn")

        with gr.Row(elem_id="advanced-options"):
            samples = gr.Slider(label="Images", minimum=1, maximum=4, value=4, step=1)
            # steps = gr.Slider(label="Steps", minimum=1, maximum=50, value=45, step=1)
            # scale = gr.Slider(
            #     label="Guidance Scale", minimum=0, maximum=50, value=7.5, step=0.1
            # )
            # seed = gr.Slider(
            #     label="Seed",
            #     minimum=0,
            #     maximum=2147483647,
            #     step=1,
            #     randomize=True,
            # )
        text.submit(infer, inputs=[text, samples], outputs=gallery)
        btn.click(infer, inputs=[text, samples], outputs=gallery)
        advanced_button.click(
            None,
            [],
            text,
        )

   
      
block.launch(share=True)

Mounted at /content/drive
/content/drive/My Drive/fashion
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`

Setting up a public link... we have recently upgraded the way public links are generated. If you encounter any problems, please report the issue and downgrade to gradio version 3.13.0
.
Running on public URL: https://10d99af1-9ccf-458a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
